In [38]:
import pandas as pd
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
import sliding_window
from sklearn.ensemble import RandomForestClassifier

In [39]:
dataset = stream.iter_csv(
            './bpic17_length28.csv',
            )

totallength = len(list(dataset))

In [40]:
dataset = stream.iter_csv(
            './bpic17_length28.csv',
#             drop=['Complete Timestamp'],
            )
enctype = 'Index-base'

In [41]:
key_pair = {
# 'Case ID':'caseid',
# 'Activity':'activity',
# 'Resource':'resource',
# 'Complete Timestamp':'ts',
}
catatars= ['activity','resource']

case_dict ={}
training_models ={}

casecount = 0
rowcounter = 0
resultdict ={}
acc_dict ={}

prediction_result = {}
graceperiod_finish=0
finishedcases = set()

In [42]:
# Sliding window for training setting
window_size = 20
training_window = sliding_window.training_window(window_size)


In [43]:
def display_progress(row_counting, total_length, interval=500):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount))

In [44]:
for x,y in dataset:
    display_progress(rowcounter, totallength)
    rowcounter +=1
    
    utils.dictkey_chg(x, key_pair)
    # Event stream change dictionary keys
    x['ts'] = x['ts'][:-4]
    
    # Check label possible
    
    # Initialize case by prefix length
    caseid = x['caseid']
    x.pop('caseid')
    
    case_bin = prefix_bin(caseid, x)
    
    if caseid not in list(case_dict.keys()):
        case_dict[caseid] = []
        case_bin.set_prefix_length(1)
        
    elif caseid in finishedcases:
        continue
    
    else:
        case_bin.set_prefix_length(len(case_dict[caseid])+1)
        case_bin.set_prev_enc(case_dict[caseid][-1])
    
    # Encode event and cases and add to DB
    ts = case_bin.event['ts']
    case_bin.update_encoded(catattrs=catatars,enctype=enctype)
    
    # Set current activity as outcome of previous event
    if case_bin.prefix_length != 1:
        case_bin.prev_enc.update_truelabel(x['activity'])

    
    if x['activity'] == 'End':
        training_window.update_window({caseid: case_dict[caseid]})
        case_dict.pop(caseid)
    else:
        case_dict[caseid].append(case_bin)
    
    

0.0 % Case finished: 0
6.22 % Case finished: 0
12.44 % Case finished: 0
18.67 % Case finished: 0
24.89 % Case finished: 0
31.11 % Case finished: 0
37.33 % Case finished: 0
43.55 % Case finished: 0
49.78 % Case finished: 0
56.0 % Case finished: 0
62.22 % Case finished: 0
68.44 % Case finished: 0
74.66 % Case finished: 0
80.89 % Case finished: 0
87.11 % Case finished: 0
93.33 % Case finished: 0
99.55 % Case finished: 0


In [47]:
x_train = []
y_train = []
for t in training_window.getAllitems():
    x_train.append(list(t.values())[0][5].encoded)
    y_train.append(list(t.values())[0][5].true_label)

In [50]:
print(x_train)

[{'duration_1': 0, 'cumduration_1': 0, 'activity_1 A_Create Application': 1, 'resource_1 User_5': 1, 'duration_2': 0.0, 'cumduration_2': 0.0, 'activity_2 W_Complete application': 1, 'resource_2 User_5': 1, 'duration_3': 0.0, 'cumduration_3': 0.0, 'activity_3 W_Complete application': 1, 'resource_3 User_5': 1, 'duration_4': 0.0, 'cumduration_4': 0.0, 'activity_4 A_Concept': 1, 'resource_4 User_5': 1, 'duration_5': 283.0, 'cumduration_5': 283.0, 'activity_5 A_Accepted': 1, 'resource_5 User_5': 1, 'duration_6': 153.0, 'cumduration_6': 436.0, 'activity_6 O_Create Offer': 1, 'resource_6 User_5': 1}, {'duration_1': 0, 'cumduration_1': 0, 'activity_1 A_Create Application': 1, 'resource_1 User_1': 1, 'duration_2': 1.0, 'cumduration_2': 1.0, 'activity_2 A_Submitted': 1, 'resource_2 User_1': 1, 'duration_3': 0.0, 'cumduration_3': 1.0, 'activity_3 W_Handle leads': 1, 'resource_3 User_1': 1, 'duration_4': 80.0, 'cumduration_4': 81.0, 'activity_4 W_Handle leads': 1, 'resource_4 User_1': 1, 'duratio

In [56]:
x_train = pd.DataFrame.from_dict(x_train)
x_train = x_train.fillna(0)
print(x_train)

    duration_1  cumduration_1  activity_1 A_Create Application  \
0            0              0                                1   
1            0              0                                1   
2            0              0                                1   
3            0              0                                1   
4            0              0                                1   
5            0              0                                1   
6            0              0                                1   
7            0              0                                1   
8            0              0                                1   
9            0              0                                1   
10           0              0                                1   
11           0              0                                1   
12           0              0                                1   
13           0              0                                1   
14        

In [57]:
clf  = RandomForestClassifier()
clf.fit(x_train,y_train)

RandomForestClassifier()

In [68]:
print(clf.classes_)
print(clf.predict_proba(x_train.loc[19:]))
print(clf.predict(x_train.loc[19:]))

['A_Accepted' 'O_Create Offer' 'O_Created' 'W_Complete application']
[[0.06 0.   0.91 0.03]]
['O_Created']


In [62]:
print(clf.score(x_train,y_train))

1.0
